<a href="https://colab.research.google.com/github/guillermodangelo/tesis/blob/master/matriz_de_distancias_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas

In [0]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import requests
import urllib
import json

In [3]:
# carga capa localidades INE pt
localidad = gpd.read_file('https://raw.githubusercontent.com/guillermodangelo/tesis/master/capas/ine_localidades.gpkg')
localidad.CODLOC = localidad.CODLOC.astype(int)

# filtra capitales departamentales de las localidades INE
capital = localidad[localidad.CAPITAL==True].reset_index(drop=True)
capital.head()

,DEPTO,LOCALIDAD,CODLOC,NOMBDEPTO,NOMBLOC,CDEPTO_ISO,CLOC_ISO,ETIQUETA,CAPITAL,geometry
0,2,220,2220,ARTIGAS,ARTIGAS,UYAR,UYARART,Artigas,True,POINT (549273.219 6633915.063)
1,3,320,3320,CANELONES,CANELONES,UYCA,UYCACAN,Canelones,True,POINT (565955.641 6179409.202)
2,4,220,4220,CERRO LARGO,MELO,UYCL,UYCLMLO,Melo,True,POINT (766475.520 6415360.379)
3,5,320,5320,COLONIA,COLONIA DEL SACRAMENTO,UYCO,UYCOCDS,Colonia,True,POINT (422937.981 6187566.893)
4,6,220,6220,DURAZNO,DURAZNO,UYDU,UYDUDNO,Durazno,True,POINT (544395.697 6306205.765)


In [4]:
# obtiene coordenadas en proyección geográfica (EPSG 4326)
lon = capital['geometry'].to_crs(epsg=4326).x
lat = capital['geometry'].to_crs(epsg=4326).y
capital_repro = pd.concat( [ pd.DataFrame( list(lat), columns=['lat'] ), pd.DataFrame( list(lon), columns=['lon'] ) ], axis=1, sort=False)

# concatena en un sólo string
capital_repro['latlon'] = capital_repro.apply(lambda x:'%s,%s' % (x['lat'],x['lon']),axis=1)

# concatena ambos df
df_capital = pd.concat([capital, capital_repro], axis=1, ignore_index=False)
df_capital.head()

,DEPTO,LOCALIDAD,CODLOC,NOMBDEPTO,NOMBLOC,CDEPTO_ISO,CLOC_ISO,ETIQUETA,CAPITAL,geometry,lat,lon,latlon
0,2,220,2220,ARTIGAS,ARTIGAS,UYAR,UYARART,Artigas,True,POINT (549273.219 6633915.063),-30.425838,-56.486918,"-30.42583821419782,-56.48691836714405"
1,3,320,3320,CANELONES,CANELONES,UYCA,UYCACAN,Canelones,True,POINT (565955.641 6179409.202),-34.524882,-56.281348,"-34.5248820942228,-56.28134757794109"
2,4,220,4220,CERRO LARGO,MELO,UYCL,UYCLMLO,Melo,True,POINT (766475.520 6415360.379),-32.366966,-54.167977,"-32.366965911958815,-54.16797657500114"
3,5,320,5320,COLONIA,COLONIA DEL SACRAMENTO,UYCO,UYCOCDS,Colonia,True,POINT (422937.981 6187566.893),-34.450550,-57.838920,"-34.45055037560706,-57.838919942012616"
4,6,220,6220,DURAZNO,DURAZNO,UYDU,UYDUDNO,Durazno,True,POINT (544395.697 6306205.765),-33.382502,-56.522684,"-33.38250225260489,-56.5226838660229"


In [5]:
# realiza el producto cartesiano para obtener todas las combinaciones posibles de localidades
df_capital_coords = df_capital[['CODLOC', 'latlon']]
df_capital_coords['key']=0

df_prod_cartesiano = df_capital_coords.merge(df_capital_coords, on='key', how='outer')
df_prod_cartesiano.drop(columns=['key'], inplace=True)
df_prod_cartesiano.rename(columns={'CODLOC_x': 'cod_ori', 'latlon_x': 'latlon_ori',
                                       'CODLOC_y': 'cod_des', 'latlon_y': 'latlon_des'}, inplace=True)

# elimina misma cod origen-destino
df_prod_cartesiano = df_prod_cartesiano[df_prod_cartesiano['cod_ori'] != df_prod_cartesiano['cod_des']]

# concatena en un código
df_prod_cartesiano['cod'] = (df_prod_cartesiano['cod_ori'].astype(str) + df_prod_cartesiano['cod_des'].astype(str)).astype('int')

print(df_prod_cartesiano.shape[0])
df_prod_cartesiano.head(3)

342


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,cod_ori,latlon_ori,cod_des,latlon_des,cod
1,2220,"-30.42583821419782,-56.48691836714405",3320,"-34.5248820942228,-56.28134757794109",22203320
2,2220,"-30.42583821419782,-56.48691836714405",4220,"-32.366965911958815,-54.16797657500114",22204220
3,2220,"-30.42583821419782,-56.48691836714405",5320,"-34.45055037560706,-57.838919942012616",22205320


In [0]:
# matriz de distancia entre capitales departamentales de departamentos

# define la función obtenerDistancia que usa la API Distance Matrix de Google

def obtenerDistancia(ori, dest, API_KEY):
    '''
    Esta funcion devuelve distancia en km entre un o varios par(es) de coordenadas
    geográficas, utilizando la API de Google Distance Matrix.
    '''
    APIurl = """https://maps.googleapis.com/maps/api/distancematrix/json?
                 units=metric&mode=walking&origins=""" + ori +\
                 "&destinations=" + dest + "&key=" + API_KEY
    #Obtener la respuesta y cargar la representacion en un string
    response = requests.get(APIurl)
    geodataString = response.text
    geodata = json.loads(geodataString)
    # si la API no devulve resultados, la distancia será 'None'
    if geodata['rows'][0]['elements'][0]['status']=='ZERO_RESULTS':
        distancia = None
    else:
        distancia = geodata['rows'][0]['elements'][0]['distance']['value']
    return distancia

In [7]:
# setea la API KEY
API_KEY = 'API KEY proporcionada por los web services de Google Map Platform'

# ordena y resetea el índice
df_prod_cartesiano.sort_values(by=['cod_ori', 'cod_des'], inplace=True)
df_prod_cartesiano.reset_index(drop=True, inplace=True)

print(df_prod_cartesiano.shape[0])
df_prod_cartesiano.head(3)

342


,cod_ori,latlon_ori,cod_des,latlon_des,cod
0,1020,"-34.83459368462574,-56.200727322283996",2220,"-30.42583821419782,-56.48691836714405",10202220
1,1020,"-34.83459368462574,-56.200727322283996",3320,"-34.5248820942228,-56.28134757794109",10203320
2,1020,"-34.83459368462574,-56.200727322283996",4220,"-32.366965911958815,-54.16797657500114",10204220


In [0]:
# son 342 díadas de localdiades
# for loop para obtener una lista con las distancias entre localidades
# aplica la función obtenerDistancia para combinaciones de localidades

distancias=[]

for x in range(0, df_prod_cartesiano.shape[0]):
  distancias.append(obtenerDistancia( df_prod_cartesiano.latlon_ori[x] , df_prod_cartesiano.latlon_des[x] , API_KEY))

In [10]:
print(len(distancias))

342


18

In [11]:
# pega con el df original
# se conviernte en matriz de distancias, data frame
df_distancias_google = pd.concat([df_prod_cartesiano.reset_index(drop=True), pd.DataFrame(distancias, columns=['distancia'])], axis=1)
df_distancias_google.head()

,cod_ori,latlon_ori,cod_des,latlon_des,cod,distancia
0,1020,"-34.83459368462574,-56.200727322283996",2220,"-30.42583821419782,-56.48691836714405",10202220,575727
1,1020,"-34.83459368462574,-56.200727322283996",3320,"-34.5248820942228,-56.28134757794109",10203320,36864
2,1020,"-34.83459368462574,-56.200727322283996",4220,"-32.366965911958815,-54.16797657500114",10204220,381442
3,1020,"-34.83459368462574,-56.200727322283996",5320,"-34.45055037560706,-57.838919942012616",10205320,172714
4,1020,"-34.83459368462574,-56.200727322283996",6220,"-33.38250225260489,-56.5226838660229",10206220,175468
